In [1]:
import pandas as pds
import rdflib
from rdflib import URIRef, BNode, Literal, Graph, Namespace, RDF, RDFS, OWL
from rdflib.plugins.sparql.processor import SPARQLResult
from SPARQLWrapper import SPARQLWrapper
from pandasql import sqldf
from typing import Optional, List, Any
pysqldf = lambda q: sqldf(q, globals())

## load data

In [2]:
patients = pds.read_csv('../data/patients.csv')
providers = pds.read_csv('../data/providers.csv')
procedures = pds.read_csv('../data/procedures.csv')

test with simple sql query

In [3]:
pysqldf('select * from procedures')

,proc_id,proc_date,patient_id,provider_id,tooth_num,proc_code
0,1,2020-01-01,1001,1,1,d2300
1,2,2020-02-01,1002,2,2,d2400
2,3,2020-03-01,1003,3,3,d2500
3,4,2020-04-01,1004,1,4,d2600
4,5,2020-05-01,1005,2,5,d2700
5,6,2020-06-01,1006,3,6,d2800


## helper functions
* add_spo: shortcut for add subject, predicate, object URIRefs to graph
* add_spo: shortcut for add subject, predicate URIRefs and literal value to graph
* sparql_results_to_df: converts sparql results into a dataframe
* sparql_query_to_df: queries the graph and returns the results as a dataframe
* add_table_metadata_to_graph: adds instances of tables and fields to graph
* add_enums_to_graph: adds instances of enumerated values to graph

In [4]:
def add_spo(graph: Graph, subj: Any, predicate: Any, obj: Any) -> Graph:
    graph.add((URIRef(subj), URIRef(predicate), URIRef(obj)))
    return graph

In [5]:
def add_spv(graph: Graph, subj: Any, predicate: Any, val: Any) -> Graph:
    graph.add((URIRef(subj), URIRef(predicate), Literal(val)))
    return graph

In [6]:
def sparql_results_to_df(results: SPARQLResult, graph=Optional[Graph]) -> pds.DataFrame:
    def set_value(x):
        if x is None:
            return None
        elif graph is not None:
            for n in graph.namespaces():
                # each is a tuple of form (<prefix>, URIRef(...))
                # e.g., ('dc', rdflib.term.URIRef('http://purl.org/dc/elements/1.1/'))
                if str(x).startswith(str(n[1])):
                    # replace uri with prefix
                    return str(x).replace(n[1], n[0])
                
            # if it makes it here, no replacements occurred
            return x.toPython()
        else:
            return x.toPython()

    return \
        pds.DataFrame(
            data=([set_value(x) for x in row] for row in results),
            columns=[str(x) for x in results.vars]
        )

In [7]:
def sparql_query_to_df(query: str, graph: Graph, use_ns=True) -> pds.DataFrame:
    results = graph.query(query)
    if use_ns:
        return sparql_results_to_df(results, graph)
    else:
        return sparql_results_to_df(results, None)

In [8]:
def add_table_metadata_to_graph(table: pds.DataFrame, 
                                table_name: str, 
                                graph: Graph, 
                                table_ns: Namespace, 
                                field_ns: Namespace, 
                                property_ns: Namespace) -> Graph:
    # add table instance to graph
    table_uri = table_ns[f'/{table_name}']
    graph = add_spo(graph, table_uri, RDF.type, table_ns)
    graph = add_spv(graph, table_uri, RDFS.label, table_name)
        
    # add each of the tables fields to graph as instances of fields
    for field_name in table.columns:
        field_name = f'{table_name}.{field_name}' # prepend table name to field name
        uri = URIRef(field_ns[f'/{field_name}'])
        graph = add_spo(graph, uri, RDF.type, field_ns)
        grpah = add_spo(graph, uri, property_ns.member_of, table_uri)
        graph = add_spv(graph, uri, RDFS.label, field_name)
    
    return graph

In [9]:
def add_enums_to_graph(enums: List, 
                       table_name: str, 
                       field_name: str, 
                       graph: Graph, 
                       enum_ns: Namespace, 
                       base_ns: Namespace) -> Graph:

    field_name = f'{table_name}.{field_name}' # prepend table name to field name
    for enum in enums:
        # build uri
        uri = URIRef(enum_ns[f'/{field_name}#{enum}'])

        # add instances
        # Note: the literal value is added to the graph as well
        graph = add_spo(graph, uri, RDF.type, enum_ns)
        graph = add_spv(graph, uri, base_ns.has_value, enum)
        graph = add_spv(graph, uri, RDFS.label, f'{field_name} {enum}')

        # enums constrain values in fields, so add this informaton the graph
        field = base_ns[f'/field/{field_name}']
        graph = add_spo(graph, uri, base_ns.defines_values_in, field)
        
    return graph

## load ontology

In [10]:
g = Graph()
g.parse('../ontology/data-field-punning.ttl', format='ttl')
g.bind(":", Namespace("https://data-field-punning.owl/")) # you can also use g.namespace_manager.bind(...)
g.bind("field:", Namespace("https://data-field-punning.owl/field/"))
g.bind("table:", Namespace("https://data-field-punning.owl/table/"))
g.bind("enum:", Namespace("https://data-field-punning.owl/enumerated_value/"))

add some namespaces to use as shortcuts

In [11]:
ns = Namespace("https://data-field-punning.owl/")
field_ns = Namespace(ns.field)
table_ns = Namespace(ns.table)
enum_ns = Namespace(ns.enumerated_value)

test simple sparql query

In [12]:
q = """
select ?cls ?cls_label where {
  ?cls a owl:Class
  optional {?cls rdfs:label ?cls_label}
}
"""
sparql_query_to_df(q, g).head() # note: I only display the first 5 results

,cls,cls_label
0,:canine,canine
1,:crown_restoration,crown restoration
2,:data_value,data value
3,:dentist,dentist
4,:entity,None


## add table and field instances to graph

In [13]:
g = add_table_metadata_to_graph(patients, 'patients', g, table_ns, field_ns, ns)
g = add_table_metadata_to_graph(providers, 'providers', g, table_ns, field_ns, ns)
g = add_table_metadata_to_graph(procedures, 'procedures', g, table_ns, field_ns, ns)

query to check that instaces where added

In [14]:
q = """
prefix : <https://data-field-punning.owl/>
select ?field ?type ?field_name ?table ?table_name where {
  ?field a :field;
         rdfs:label ?field_name;
         rdf:type ?type;
         :member_of ?table .
  ?table rdfs:label ?table_name .
}
"""
sparql_query_to_df(q, g)

,field,type,field_name,table,table_name
0,:field/patients.patient_id,:field,patients.patient_id,:table/patients,patients
1,:field/patients.name,:field,patients.name,:table/patients,patients
2,:field/patients.gender,:field,patients.gender,:table/patients,patients
3,:field/patients.dob,:field,patients.dob,:table/patients,patients
4,:field/providers.provider_id,:field,providers.provider_id,:table/providers,providers
5,:field/providers.name,:field,providers.name,:table/providers,providers
6,:field/procedures.proc_id,:field,procedures.proc_id,:table/procedures,procedures
7,:field/procedures.proc_date,:field,procedures.proc_date,:table/procedures,procedures
8,:field/procedures.patient_id,:field,procedures.patient_id,:table/procedures,procedures
9,:field/procedures.provider_id,:field,procedures.provider_id,:table/procedures,procedures


## add enumerated values
The values in `patients.gender` and `procedures.proc_code` are enums. Let's add them to ontology shema.  
Note: For demonstration purposes, I've made enums url safe. In a real-world scenario, the enums would need to be url encoded.

In [15]:
genders = list(pysqldf("select distinct gender from patients")['gender'])
proc_codes = list(pysqldf("select distinct proc_code from procedures")['proc_code'])

In [16]:
g = add_enums_to_graph(genders, 'patients', 'gender', g, enum_ns, ns)
g = add_enums_to_graph(proc_codes, 'procedures', 'proc_code', g, enum_ns, ns)

query the enums added to graph

In [17]:
q = """
prefix : <https://data-field-punning.owl/>
select ?enum ?label ?value ?defines where {
  ?enum a :enumerated_value;
    rdfs:label ?label;
    :has_value ?value;
    :defines_values_in ?defines .
}
"""
sparql_query_to_df(q, g)

,enum,label,value,defines
0,:enumerated_value/patients.gender#M,patients.gender M,M,:/field/patients.gender
1,:enumerated_value/patients.gender#F,patients.gender F,F,:/field/patients.gender
2,:enumerated_value/procedures.proc_code#d2300,procedures.proc_code d2300,d2300,:/field/procedures.proc_code
3,:enumerated_value/procedures.proc_code#d2400,procedures.proc_code d2400,d2400,:/field/procedures.proc_code
4,:enumerated_value/procedures.proc_code#d2500,procedures.proc_code d2500,d2500,:/field/procedures.proc_code
5,:enumerated_value/procedures.proc_code#d2600,procedures.proc_code d2600,d2600,:/field/procedures.proc_code
6,:enumerated_value/procedures.proc_code#d2700,procedures.proc_code d2700,d2700,:/field/procedures.proc_code
7,:enumerated_value/procedures.proc_code#d2800,procedures.proc_code d2800,d2800,:/field/procedures.proc_code


## add what the data represents
The data in the tables represent things in the world. We need to connect the data to their representations.  
I created a simple mapping between IRIs and the classes represented by the them. This could also be done using a `robot` template or `SSSOM` mapping file.  
Some of mappings are at the field level. For example, the patient_id field represents a patient in general. Other mappings are a the level of enumaterated values. For example, the value "F" in the patient.gender field represents a female.  
**Note**: This mapping involves punning the classes as inviduals b/c the represents object property holds between individuals.

In [18]:
for idx, iri, entity in pds.read_csv('../data/data_representations.csv').itertuples():
    uri = URIRef(iri)
    entity_uri = URIRef(entity)
    g.add((uri, ns.represents, entity_uri))

In [19]:
q = """
prefix : <https://data-field-punning.owl/>
select ?uri ?label ?represents where {
    ?uri :represents ?represents .
    optional {
      ?uri rdfs:label ?label
    }
}
"""
sparql_query_to_df(q, g)

,uri,label,represents
0,:field/patients.patient_id,patients.patient_id,:patient
1,:field/procedures.patient_id,procedures.patient_id,:patient
2,:field/providers.provider_id,providers.provider_id,:dentist
3,:field/procedures.provider_id,procedures.provider_id,:dentist
4,:field/procedures.proc_id,procedures.proc_id,:procedure
5,:field/procedures.proc_code,procedures.proc_code,:procedure
6,:field/procedures.tooth_num,procedures.tooth_num,:tooth
7,:enumerated_value/patients.gender#M,patients.gender M,:male_patient
8,:enumerated_value/patients.gender#F,patients.gender F,:female_patient
9,:enumerated_value/procedures.proc_code#d2300,procedures.proc_code d2300,:crown_restoration


## use representations to form sql queries

example 1: get every field name that represents a person.  
note: This finds all fields that represent a subclass of `person`.

In [187]:
q = """
prefix : <https://data-field-punning.owl/>
select distinct ?table_name ?field_name (group_concat(?cls_name) as ?cls_names) where {
    ?cls rdfs:subClassOf :person;
         rdfs:label ?cls_label .
         
    ?field a :field;
        :represents ?cls;
        rdfs:label ?field_name;
        :member_of [a :table; rdfs:label ?table_name] .
        
    bind(replace(?cls_label, " ", "_") as ?cls_name)
}
group by ?table_name ?field_name
"""
field_df = sparql_query_to_df(q, g)
field_df

,table_name,field_name,cls_names
0,patients,patients.patient_id,patient
1,providers,providers.provider_id,dentist
2,procedures,procedures.patient_id,patient
3,procedures,procedures.provider_id,dentist


build sql query

In [188]:
table_type_dict = {}

for idx, table_name, field_name, cls_name in field_df.itertuples():
    print(table_name, field_name, cls_name)
    

patients patients.patient_id patient
providers providers.provider_id dentist
procedures procedures.patient_id patient
procedures procedures.provider_id dentist


In [191]:
schema_dict = {
    'field_names': [],
    'table_names': [],
    'type_matches': []
}

joins = []
type_to_field = {}
temp_dict = {}

for group in field_df.groupby('table_name'):
    group_table = group[0]
    group_df = group[1]
    schema_dict['field_names'].extend(list(group_df['field_name']))
    schema_dict['table_names'].append(group_table)

for group in field_df.groupby('cls_names'):
    group_cls = group[0]
    group_df = group[1]
    # print(list(zip(list(group_df['table_name']), list(group_df['field_name']))))
    type_to_field[group_cls] = list(group_df['field_name'])
    temp_dict[group_cls] = list(zip(list(group_df['table_name']), list(group_df['field_name'])))
    # schema_dict['type_matches'].append(list(group_df['field_name']))
    # schema_dict['type_matches'].append(list(zip(list(group_df['table_name']), list(group_df['field_name']))))
    
# print('select', ', '.join(schema_dict['field_names']))
# print('from',  ', '.join(schema_dict['table_names']))
type_to_field
# temp_dict

{'dentist': ['providers.provider_id', 'procedures.provider_id'],
 'patient': ['patients.patient_id', 'procedures.patient_id']}

In [192]:
type_to_field['dentist']= ['providers.provider_id', 'procedures.provider_id', 'patients.provider_id']
type_pairs = []
for k,v in type_to_field.items():
    if len(v) > 1:
        # print(list(accumulate(v, lambda f1, f2: f"on {f1} = {f2}"))) # doesnt' work
        for i in range(len(v) - 1):
            # print("on", v[i], "=", v[i + 1])
            type_pairs.append((v[1], v[i + 1]))
type_pairs           

[('procedures.provider_id', 'procedures.provider_id'),
 ('procedures.provider_id', 'patients.provider_id'),
 ('procedures.patient_id', 'procedures.patient_id')]

In [193]:
temp_dict

{'dentist': [('providers', 'providers.provider_id'),
  ('procedures', 'procedures.provider_id')],
 'patient': [('patients', 'patients.patient_id'),
  ('procedures', 'procedures.patient_id')]}

In [194]:
# print("from", temp_dict[0][0])
for k,v in temp_dict.items():
    if len(v) > 1:
        for i in range(len(v) - 1):
            left_field = v[i][1]
            right_field = v[i + 1][1]
            print("inner join",  v[i + 1][0], "on", left_field, "=", right_field)

inner join procedures on providers.provider_id = procedures.provider_id
inner join procedures on patients.patient_id = procedures.patient_id


In [195]:
foo = {}
foo[(1, 2)] = 'bar'

In [196]:
foo

{(1, 2): 'bar'}

In [197]:
temp = schema_dict['type_matches'].copy()

In [198]:
temp

[]

In [199]:
from_clause = f"from {schema_dict['table_names'][0]}"
from_clause

'from patients'

In [200]:
if len(schema_dict['table_names']) > 1:
    for name in schema_dict['table_names'][1:]:
        print(f"inner join {name}")

inner join procedures
inner join providers


In [201]:
for i, group in enumerate(field_df.groupby('table_name')):
    print(f'group {i}', group)

group 0 ('patients',   table_name           field_name cls_names
0   patients  patients.patient_id   patient)
group 1 ('procedures',    table_name              field_name cls_names
2  procedures   procedures.patient_id   patient
3  procedures  procedures.provider_id   dentist)
group 2 ('providers',   table_name             field_name cls_names
1  providers  providers.provider_id   dentist)


In [202]:
table_dict = {}
for idx, group in enumerate(field_df.groupby('table_name')):
    group_id  = group[0]
    group_df = group[1]
    
    field_names = list(group_df['field_name'])
    if idx ==0:
        table_dict[group_id] = {'clause': f'from {group_id}', 'field_names': field_names}
    else:
        table_dict[group_id] = {'clause': f'inner join {group_id}', 'field_names': field_names}
    # print(group_df)
table_dict

{'patients': {'clause': 'from patients',
  'field_names': ['patients.patient_id']},
 'procedures': {'clause': 'inner join procedures',
  'field_names': ['procedures.patient_id', 'procedures.provider_id']},
 'providers': {'clause': 'inner join providers',
  'field_names': ['providers.provider_id']}}

In [203]:
# field_df.to_dict(orient='list')

In [204]:
# field_df.to_dict(orient='series')

In [205]:
# field_df.to_dict(orient='split')

In [206]:
# field_df.to_dict(orient='tight')

In [207]:
field_df.to_dict(orient='records')

[{'table_name': 'patients',
  'field_name': 'patients.patient_id',
  'cls_names': 'patient'},
 {'table_name': 'providers',
  'field_name': 'providers.provider_id',
  'cls_names': 'dentist'},
 {'table_name': 'procedures',
  'field_name': 'procedures.patient_id',
  'cls_names': 'patient'},
 {'table_name': 'procedures',
  'field_name': 'procedures.provider_id',
  'cls_names': 'dentist'}]

In [208]:
# field_df.to_dict(orient='index')

In [209]:
field_df.set_index('table_name').loc['procedures'][['field_name', 'cls_names']]

,field_name,cls_names
table_name,,
procedures,procedures.patient_id,patient
procedures,procedures.provider_id,dentist


In [210]:
field_df.set_index('field_name').to_dict(orient='index')

{'patients.patient_id': {'table_name': 'patients', 'cls_names': 'patient'},
 'providers.provider_id': {'table_name': 'providers', 'cls_names': 'dentist'},
 'procedures.patient_id': {'table_name': 'procedures', 'cls_names': 'patient'},
 'procedures.provider_id': {'table_name': 'procedures',
  'cls_names': 'dentist'}}

In [211]:
filed_to_table_dict = field_df[['field_name', 'table_name']].set_index('field_name').to_dict(orient='index')
filed_to_table_dict

{'patients.patient_id': {'table_name': 'patients'},
 'providers.provider_id': {'table_name': 'providers'},
 'procedures.patient_id': {'table_name': 'procedures'},
 'procedures.provider_id': {'table_name': 'procedures'}}

In [220]:
multi = field_df.set_index(['table_name', 'field_name']).sort_index()
multi

cls_names
table_name field_name                      
patients   patients.patient_id      patient
procedures procedures.patient_id    patient
           procedures.provider_id   dentist
providers  providers.provider_id    dentist

In [224]:
multi.loc[('procedures', 'procedures.patient_id'), : ]

cls_names    patient
Name: (procedures, procedures.patient_id), dtype: object

In [243]:
multi.loc[('procedures', slice(None)), : ]

cls_names
table_name field_name                      
procedures procedures.patient_id    patient
           procedures.provider_id   dentist

In [230]:
cmulti = field_df.set_index(['cls_names', 'field_name']).sort_index()
cmulti

table_name
cls_names field_name                        
dentist   procedures.provider_id  procedures
          providers.provider_id    providers
patient   patients.patient_id       patients
          procedures.patient_id   procedures

In [231]:
list(cmulti.groupby('table_name'))

[('patients',
                                table_name
  cls_names field_name                    
  patient   patients.patient_id   patients),
 ('procedures',
                                    table_name
  cls_names field_name                        
  dentist   procedures.provider_id  procedures
  patient   procedures.patient_id   procedures),
 ('providers',
                                  table_name
  cls_names field_name                      
  dentist   providers.provider_id  providers)]

In [219]:
field_df.set_index(['cls_names', 'field_name']).index.values

array([('patient', 'patients.patient_id'),
       ('dentist', 'providers.provider_id'),
       ('patient', 'procedures.patient_id'),
       ('dentist', 'procedures.provider_id')], dtype=object)

In [215]:
field_df.set_index(['table_name','field_name']).to_dict(orient='index')

{('patients', 'patients.patient_id'): {'cls_names': 'patient'},
 ('providers', 'providers.provider_id'): {'cls_names': 'dentist'},
 ('procedures', 'procedures.patient_id'): {'cls_names': 'patient'},
 ('procedures', 'procedures.provider_id'): {'cls_names': 'dentist'}}

In [216]:
import itertools

In [217]:
list(itertools.product(['providers.provider_id', 'procedures.provider_id', 'patients.provider_id']))

[('providers.provider_id',),
 ('procedures.provider_id',),
 ('patients.provider_id',)]

In [218]:
list(itertools.combinations(['providers.provider_id', 'procedures.provider_id', 'patients.provider_id'], 2))

[('providers.provider_id', 'procedures.provider_id'),
 ('providers.provider_id', 'patients.provider_id'),
 ('procedures.provider_id', 'patients.provider_id')]

In [226]:
multi.to_dict(orient='index')

{('patients', 'patients.patient_id'): {'cls_names': 'patient'},
 ('procedures', 'procedures.patient_id'): {'cls_names': 'patient'},
 ('procedures', 'procedures.provider_id'): {'cls_names': 'dentist'},
 ('providers', 'providers.provider_id'): {'cls_names': 'dentist'}}

In [245]:
list(itertools.combinations(['providers.provider_id',], 2))

[]

In [247]:
' = '.join(('1', '2'))

'1 = 2'

In [258]:
multi.index.values

array([('patients', 'patients.patient_id'),
       ('procedures', 'procedures.patient_id'),
       ('procedures', 'procedures.provider_id'),
       ('providers', 'providers.provider_id')], dtype=object)

In [260]:
list(itertools.combinations(multi.index.values, 2))

[(('patients', 'patients.patient_id'),
  ('procedures', 'procedures.patient_id')),
 (('patients', 'patients.patient_id'),
  ('procedures', 'procedures.provider_id')),
 (('patients', 'patients.patient_id'), ('providers', 'providers.provider_id')),
 (('procedures', 'procedures.patient_id'),
  ('procedures', 'procedures.provider_id')),
 (('procedures', 'procedures.patient_id'),
  ('providers', 'providers.provider_id')),
 (('procedures', 'procedures.provider_id'),
  ('providers', 'providers.provider_id'))]

In [263]:
list(field_df.groupby(['cls_names', 'table_name']))

[(('dentist', 'procedures'),
     table_name              field_name cls_names
  3  procedures  procedures.provider_id   dentist),
 (('dentist', 'providers'),
    table_name             field_name cls_names
  1  providers  providers.provider_id   dentist),
 (('patient', 'patients'),
    table_name           field_name cls_names
  0   patients  patients.patient_id   patient),
 (('patient', 'procedures'),
     table_name             field_name cls_names
  2  procedures  procedures.patient_id   patient)]

In [262]:
list(multi.groupby('cls_names'))

[('dentist',
                                    cls_names
  table_name field_name                      
  procedures procedures.provider_id   dentist
  providers  providers.provider_id    dentist),
 ('patient',
                                   cls_names
  table_name field_name                     
  patients   patients.patient_id     patient
  procedures procedures.patient_id   patient)]

In [254]:
table_pairs = list(itertools.combinations(list(field_df['table_name']), 2))
table_pairs

[('patients', 'providers'),
 ('patients', 'procedures'),
 ('patients', 'procedures'),
 ('providers', 'procedures'),
 ('providers', 'procedures'),
 ('procedures', 'procedures')]

In [252]:
for i, (t1, t2) in enumerate(table_pairs):
    print(i, t1, t2)

0 patients providers
1 patients procedures
2 patients procedures
3 providers procedures
4 providers procedures
5 procedures procedures


In [267]:
list(field_df[['table_name', 'field_name']].values)

[array(['patients', 'patients.patient_id'], dtype=object),
 array(['providers', 'providers.provider_id'], dtype=object),
 array(['procedures', 'procedures.patient_id'], dtype=object),
 array(['procedures', 'procedures.provider_id'], dtype=object)]

In [273]:
tables_fields = field_df.set_index(['cls_names', 'field_name']).index.values.tolist()
tables_fields

[('patient', 'patients.patient_id'),
 ('dentist', 'providers.provider_id'),
 ('patient', 'procedures.patient_id'),
 ('dentist', 'procedures.provider_id')]

In [275]:
tables_fields = list(field_df[['table_name', 'field_name']].itertuples(index=None, name=None))
tables_fields

[('patients', 'patients.patient_id'),
 ('providers', 'providers.provider_id'),
 ('procedures', 'procedures.patient_id'),
 ('procedures', 'procedures.provider_id')]

In [277]:
tables_fields_pairs = itertools.combinations(tables_fields, 2)
list(tables_fields_pairs)

[(('patients', 'patients.patient_id'), ('providers', 'providers.provider_id')),
 (('patients', 'patients.patient_id'),
  ('procedures', 'procedures.patient_id')),
 (('patients', 'patients.patient_id'),
  ('procedures', 'procedures.provider_id')),
 (('providers', 'providers.provider_id'),
  ('procedures', 'procedures.patient_id')),
 (('providers', 'providers.provider_id'),
  ('procedures', 'procedures.provider_id')),
 (('procedures', 'procedures.patient_id'),
  ('procedures', 'procedures.provider_id'))]

In [290]:
temp_df = field_df.copy()
temp_df.loc[6, 'table_name'] = 'patients'
temp_df.loc[6,'field_name'] = 'patients.provider_id'
temp_df.loc[6, 'cls_names'] = 'dentist'
temp_df

,table_name,field_name,cls_names
0,patients,patients.patient_id,patient
1,providers,providers.provider_id,dentist
2,procedures,procedures.patient_id,patient
3,procedures,procedures.provider_id,dentist
6,patients,patients.provider_id,dentist


In [347]:
all_tables = list(temp_df['table_name'])
all_pairs = []
for group_name, group_df in temp_df.groupby('cls_names'):
    print('\n---', group_name, '---\n')
    
    table_names = list(group_df['table_name'])
    # print(table_names, '\n')
    # table_pairs = list(itertools.combinations(list(group_df['table_name']), 2))
    # print(table_pairs, '\n')
    tables_fields = list(group_df[['table_name', 'field_name']].itertuples(index=None, name=None))
    # print(tables_fields, '\n')
    table_field_pairs = list(itertools.combinations(tables_fields, 2))
    all_pairs.extend(table_field_pairs)
    print(table_field_pairs, '\n')
    
for i, tbl in enumerate(all_tables):
    if i == 0:
        print('from', tbl)
    else:
        # print('join', tbl)
        prev_tbl = all_tables[i - 1]
        print('  #', prev_tbl, tbl)
        
        joins = []
        for left, right in all_pairs:
            temp = (left[0], right[0])
            if prev_tbl != tbl and prev_tbl in temp and tbl in temp:
                joins.append(f'{left[1]} = {right[1]}')
        
        if len(joins) > 0:
            print(f'inner join {tbl} on ', ' and '.join(joins))


--- dentist ---

[(('providers', 'providers.provider_id'), ('procedures', 'procedures.provider_id')), (('providers', 'providers.provider_id'), ('patients', 'patients.provider_id')), (('procedures', 'procedures.provider_id'), ('patients', 'patients.provider_id'))] 


--- patient ---

[(('patients', 'patients.patient_id'), ('procedures', 'procedures.patient_id'))] 

from patients
  # patients providers
inner join providers on  providers.provider_id = patients.provider_id
  # providers procedures
inner join procedures on  providers.provider_id = procedures.provider_id
  # procedures procedures
  # procedures patients
inner join patients on  procedures.provider_id = patients.provider_id and patients.patient_id = procedures.patient_id


In [306]:
'x' in ('a', 'b')

False

In [326]:
temp_patients = patients.copy()
temp_patients['provider_id'] = [1,2,3,1,2,3]
temp_patients

,patient_id,name,gender,dob,provider_id
0,1001,Rosco,M,2001-01-01,1
1,1002,Martha,F,2002-02-02,2
2,1003,Fred,M,2003-03-03,3
3,1004,Wilma,F,2004-04-04,1
4,1005,Betty,F,2005-05-05,2
5,1006,Barney,M,2006-06-06,3


In [346]:
q = """
select temp_patients.patient_id, procedures.patient_id, procedures.provider_id, providers.provider_id
from temp_patients
inner join procedures on temp_patients.patient_id = procedures.patient_id
inner join providers on procedures.provider_id = providers.provider_id and temp_patients.provider_id = providers.provider_id
"""
sqldf(q)

,patient_id,patient_id,provider_id,provider_id
0,1001,1001,1,1
1,1002,1002,2,2
2,1003,1003,3,3
3,1004,1004,1,1
4,1005,1005,2,2
5,1006,1006,3,3
